### Implementing Karpathy's minchar vanilla RNN implementation in Shakespeare dataset.

[https://karpathy.github.io/2015/05/21/rnn-effectiveness/](https://karpathy.github.io/2015/05/21/rnn-effectiveness/)

In [8]:
import numpy as np

dataset = open('/content/shakespeare.txt.txt', 'r').read()
chars = list(set(dataset))

char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

In [2]:
# hyperparameters
hidden_size = 100
seq_length = 25
lr = 1e-1

# weights (randomly assigned)
wxh = np.random.randn(hidden_size, len(chars))*lr # input to hidden layer
whh = np.random.randn(hidden_size, hidden_size)*lr # hidden to hidden
why = np.random.randn(len(chars), hidden_size)*lr # hidden to output

#bias
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((len(chars), 1)) # output bias

In [18]:
def loss_fxn(inputs, targets, h_prev):
  xs, hs, ys, ps = {}, {}, {}, {} # input_state, hidden_state, output_state, probability_state
  hs[-1] = np.copy(h_prev)
  loss = 0

  #forward pass (computing prediction + maintaining hidden state + calculating loss)
  for t in range(len(inputs)):
    xs[t] = np.zeros((len(chars), 1)) # 1-of-k encoding
    xs[t][inputs[t]] = 1

    hs[t] = np.tanh(np.dot(wxh, xs[t]) + np.dot(whh, hs[t-1]) + bh) #hidden state
    ys[t] = np.dot(why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)

  #backward pass (computing gradients + updating weights)
  d_wxh, d_whh, d_why = np.zeros_like(wxh), np.zeros_like(whh), np.zeros_like(why) #gradients of the weights
  d_bh, d_by = np.zeros_like(bh), np.zeros_like(by)
  d_h_next = np.zeros_like(hs[0])

  for t in reversed(range(len(inputs))): # cuz backprop (bptt)
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1
    d_why += np.dot(dy, hs[t].T)
    d_by += dy
    d_h = np.dot(why.T, dy) + d_h_next # backprop into h
    d_h_raw = (1 - hs[t] * hs[t]) * d_h # backprop through tanh nonlinearity
    d_bh += d_h_raw
    d_wxh += np.dot(d_h_raw, xs[t].T)
    d_whh += np.dot(d_h_raw, hs[t-1].T)
    d_h_next = np.dot(whh.T, d_h_raw)
  for dparam in [d_wxh, d_whh, d_why, d_bh, d_by]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, d_wxh, d_whh, d_why, d_bh, d_by, hs[len(inputs)-1]

In [19]:
def sample(h, seed, n): # hidden state, random seed
  x = np.zeros((len(chars), 1))
  x[seed] = 1
  ixes = []
  for i in range(n):
    h = np.tanh(np.dot(wxh, x) + np.dot(whh, h) + bh)
    y = np.dot(why, h) + by
    p = np.exp(y) / np.sum(np.exp(y)) # exp is eulers wala exponential no.
    ix = np.random.choice(range(len(chars)), p=p.ravel())
    x = np.zeros((len(chars), 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

In [ ]:
n, p = 0, 0

mWxh, mWhh, mWhy = np.zeros_like(wxh), np.zeros_like(whh), np.zeros_like(why) #memory state
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/len(chars))*seq_length # loss at iteration 0

while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(dataset) or n == 0:
    h_prev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in dataset[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in dataset[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(h_prev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print('----\n%s\n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, d_wxh, d_whh, d_why, d_bh, d_by, h_prev = loss_fxn(inputs, targets, h_prev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print('iter %d, loss: %f' % (n, smooth_loss))

  # perform parameter update with Adagrad
  for param, dparam, mem in zip([wxh, whh, why, bh, by],
                                [d_wxh, d_whh, d_why, d_bh, d_by],
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -lr * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter

Streaming output truncated to the last 5000 lines.

MACARKE:

HINGONIA:
Cail thim,
Op: undy; byere with my hasti
----
iter 80000, loss: 48.354838
----
erpees
O
R:
Oe.

LESE BRATALL:
p-yound to to mion twell aming, your susperd;
Hake wy lers ang not favehes not hered-s fore in her nod butrer that glaunce, hall: an:
The deam her doI-neto now in to tho
----
iter 80100, loss: 48.274295
----
eairee pidour dran, 'nlatping oft, tri's must thedy there; my
plearniout we
Tondure mand not, do a gulit in not 'stertefen remare, and are wather thou thre'th theemy
Enty.

TANAXHAR:
Le
Miny bend, but
----
iter 80200, loss: 48.203476
----
he plors courry,
Or Huad. underssoncest,
Mutiod wais Sirdowire, woults
If Ged at upranders so startper fastrented tro a withre, doth it.

BRITA:
Thag louspidain sposet at visele. Gr-chat that watt.

B
----
iter 80300, loss: 48.197255
----
hancon.

MFATMATR:
-veg weed grot, me bemesseand, gut, they, bus you goling, Hadouraaen,
Lot broke.

Dy, kingsed stound hearend gany